In [1]:
import asyncio
from haskellian.asyn import syncify, then
import haskellian.either as E
from pydantic import BaseModel
from typing import Any
import pure_cv as vc
import robust_extraction as re
import robust_extraction.templates as ts
from uuid import uuid4
from moveread.core import Game, CoreAPI
from moveread.annotations.games import GameMeta
from moveread.annotations.images import ImageMeta
from moveread.sdk import MovereadAPI

In [2]:
class Result(BaseModel):
  contours: list
  corrected_url: str
  contoured_url: str

class ExtractMeta(ImageMeta):
  robust_extract_result: E.Either[Any, Result]

In [2]:
def is_group_a(game: Game):
  if game.meta is None:
    return False
  meta = GameMeta.model_validate(game.meta)
  return meta.tournament and meta.tournament.group == 'a'

api = MovereadAPI.at('/home/m4rs/.core/llobregat23/')
all_games = [
  x.unsafe()[1] for x in 
  await syncify(api.core.games.items())
]
a_games = list(filter(is_group_a, all_games))

In [4]:
async def store_results(res: re.Result, core: CoreAPI, contoured_height = 512) -> Result:
  corr = vc.encode(res.corr_img, '.jpg')
  cont = vc.draw.contours(res.contours, res.corr_img)
  cont = vc.descale_h(cont, contoured_height)
  cont = vc.encode(cont, '.jpg')
  corr_id = f'corrected/{uuid4()}'
  cont_id = f'contoured/{uuid4()}'
  await asyncio.gather(
    then(lambda x: x.unsafe(), core.blobs.insert(corr_id, corr)),
    then(lambda x: x.unsafe(), core.blobs.insert(cont_id, cont)),
  )
  return Result(contours=res.contours.tolist(), corrected_url=corr_id, contoured_url=cont_id)

In [5]:
for game in a_games:
  for i, player in enumerate(game.players):
    for j, sheet in enumerate(player.sheets):
      print(f'\r{game.id}, {i}, {j}', end='', flush=True)
      image = sheet.images[-1]
      img = (await api.core.blobs.read(image.url)).unsafe()
      mat = vc.decode(img)
      try:
        match re.extract(mat, ts.models['llobregat23']):
          case E.Left(err):
            meta = ExtractMeta(robust_extract_result=E.Left(err))
          case E.Right(ok):
            res = await store_results(ok, api.core)
            meta = ExtractMeta(robust_extract_result=E.Right(res))
      except Exception as e:
        meta = ExtractMeta(robust_extract_result=E.Left(f'Unexpected error: {e}'))
      image.meta = (image.meta or {}) | meta.model_dump(exclude_none=True)
  
  (await api.core.games.update(game.id, game)).unsafe()

a/6/61, 1, 00

/home/m4rs/mr-github/rnd/cv/robust-extraction/robust-extraction/src/robust_extraction/vectors/vectors.py:8: RuntimeWarning: invalid value encountered in divide
  return v / np.linalg.norm(v)


a/8/84, 1, 1

/home/m4rs/mr-github/rnd/cv/robust-extraction/robust-extraction/src/robust_extraction/vectors/vectors.py:8: RuntimeWarning: invalid value encountered in divide
  return v / np.linalg.norm(v)


a/5/29, 1, 0